# Measurement chains 

## Overview

**This tutorial covers:**

The `MeasurementChain` class that describes in detail how the experimental date was acquitred

**Requirements:**

Opening and navigating through WelDX files
 TODO: link tutorial, once it is merged

## Plotting the measurement chain

When working with experimental data we sometimes need more information than just the pure data itself.
For example, we might want to know how large the measurement error can be or if the data has already been proprocessed before it was written to the file.
Getting this information might be challenging because the person who conducted the experiment might be not available to you or the information is simply lost because nobody remembers and it hasn't been documented.

WelDX solves this problem by enforcing that the experimental setup is actually documented and stored during the creation of a file.
Therefore, it uses a dedicated data structure called the `Measurement`.
Let's get it out of our example file and explore the information it provides.

As always, we get us a quick overview of the file content to find out how to access the data:

In [ ]:
from weldx import WeldxFile

wxfile = WeldxFile("single_pass_weld.wx")

In [ ]:
wxfile.info()

From the file content we see, that thera actually multiple `Measurement` objects stored under the key `measurements`.
Let's extract the list and print the names of all measurements by using the `name` attribute:

In [ ]:
measurements = wxfile["measurements"]
for measurement in measurements:
    print(measurement.name)

In [ ]:
measurements[0].measurement_chain.plot()